### 1. Посчитаем ошибку на Кросс-Валидации и Тесте на выборке с различными признаками.

In [1]:
import pandas as pd

In [2]:
initial_data = pd.read_csv('initial_data.csv', index_col='id')

initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

initial_data = initial_data[initial_cols]

In [3]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))
initial_data = initial_data.drop('trip_duration', axis=1)

In [4]:
initial_data 

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,log_trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,6.122493
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,6.498282
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,7.661527
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,6.063785
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,6.077642
...,...,...,...,...,...,...,...
id2376096,2,4,-73.982201,40.745522,-73.994911,40.740170,6.658011
id1049543,1,1,-74.000946,40.747379,-73.970184,40.796547,6.486161
id2304944,2,1,-73.959129,40.768799,-74.004433,40.707371,6.639876


In [5]:
### Выделим test

from sklearn.model_selection import train_test_split

X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [6]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [7]:
### Замерим качество на кросс-валидации
from sklearn.linear_model import LinearRegression


losses_train = []
losses_test = []
for train_index, val_index in splitter.split(X_train):
    x_train, x_val = X_train.values[train_index],X_train.values[val_index]
    Y_train, Y_val = y_train.values[train_index],y_train.values[val_index]
    
    model = LinearRegression()
    model.fit(x_train,Y_train)
    
    losses_train.append(
        np.mean((model.predict(x_train) - Y_train)**2)
    )
    losses_test.append(
        np.mean((model.predict(x_val) - Y_val)**2)
    )

In [8]:
np.mean(losses_train)

0.6086150868728148

In [9]:
np.mean(losses_test)

0.6134219105328513

In [10]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте
model = LinearRegression()
model.fit(X_train,y_train)
np.mean((model.predict(X_test) - y_test)**2)
    


0.6056668879526306

### 2. Проделаем все то же самое, только для модели с более осознанными признаками

In [11]:
processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [12]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,
id2875421,1,930.399753,0,455.0,1.500479
id2377394,0,930.399753,0,663.0,1.807119
id3858529,1,930.399753,0,2124.0,6.392080
id3504673,1,930.399753,0,429.0,1.487155
id2181028,1,930.399753,0,435.0,1.189925


In [13]:
### Также замеряем MSLE.
import numpy as np

processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

In [14]:
X_2 = processed_data.drop('log_trip_duration', axis=1)
y_2 = processed_data['log_trip_duration']

In [15]:
### Важно! на валидации и на тесте, не шаффлим данные заново!
### Берем те же индексы, что и на прошлой, 
### поскольку данные из одной и той же таблицы

test_indexes = X_test.index
train_indexes = X_train.index

X_train_2 = X_2[X_2.index.isin(train_indexes)]
y_train_2 = y_2[y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
y_test_2 = y_2[y_2.index.isin(test_indexes)]

In [16]:
### Замерим качество на кросс-валидации

losses_train_2 = []
losses_test_2 = []
for train_index, val_index in splitter.split(X_train_2):
    x_train, x_val = X_train_2.values[train_index],X_train_2.values[val_index]
    Y_train, Y_val = y_train_2.values[train_index],y_train_2.values[val_index]
    
    model = LinearRegression()
    model.fit(x_train,Y_train)
    
    losses_train_2.append(
        np.mean((model.predict(x_train) - Y_train)**2)
    )
    losses_test_2.append(
        np.mean((model.predict(x_val) - Y_val)**2)
    )

In [17]:
np.mean(losses_train_2)

0.42868799007351904

In [18]:
np.mean(losses_test_2)

0.4313257704232948

In [19]:
### Теперь построим модель и замерим качество на тесте
model = LinearRegression()
model.fit(X_train_2,y_train_2)
np.mean((model.predict(X_test_2) - y_test_2)**2)

0.4074669546810294

#### Результаты получились различные. 
#### Более осознанные данные дали меньшую ошибку по оценки MSLE, что для оптимального поиска Такси-Клиента будет приемлимой первичной оценкой